In [ ]:
import pandas as pd
import spacy
import string
import regex as re
import nltk
import ast
import copy
import glob
import advertools as adv
import plotly.graph_objects as go


pd.options.display.max_colwidth = 285



from textblob import TextBlob

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


from nltk.corpus import stopwords
from urllib.parse import urlparse
from textblob import Word
from sklearn.metrics.pairwise import cosine_similarity


nlp = spacy.load("en_core_web_sm")



stemmer = nltk.SnowballStemmer("english")


# nltk.download('wordnet')
# nltk.download('omw-1.4')

## Import sentence_transformers and load pre-trained model

In [ ]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/stsb-roberta-base')

In [ ]:
def get_tweet_conversation_pair(tweet_col, conv_col):

    pairs = []
    for tweet, conv in zip(tweet_col, conv_col):
        pairs.append([tweet, conv])
    
    return pairs

In [ ]:
def add_tweet_conv_semantic_similarity(tweet_col, conv_col):
    return model.predict(get_tweet_conversation_pair(tweet_col, conv_col), show_progress_bar=True)

In [ ]:
def variance_conversation_semantic(sub_df):
    """
    input:
        sub-dataframe, containing responses of a tweet
    output:
        variance of semantic score of the tweet's conversation
    """
    if len(sub_df) > 1:
        return sub_df['semantic_similarity'].var()
    else:
        return 0